#ex02

In [1]:
%autosave 60
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

Autosaving every 60 seconds


Создадим датафрейм, назначив индексом столбец ID

In [2]:
df = pd.read_csv('../data/auto.csv', index_col='ID')

In [3]:
df

,CarNumber,Make_n_model,Refund,Fines,History
ID,,,,,
0,Y163O8161RUS,Ford Focus,2.0,3200.0,NaN
1,E432XX77RUS,Toyota Camry,1.0,6500.0,NaN
2,7184TT36RUS,Ford Focus,1.0,2100.0,NaN
3,X582HE161RUS,Ford Focus,2.0,2000.0,NaN
4,E34877152RUS,Ford Focus,2.0,6100.0,NaN
...,...,...,...,...,...
926,Y163O8161RUS,Ford Focus,2.0,1600.0,NaN
927,M0309X197RUS,Ford Focus,1.0,22300.0,NaN
928,O673E8197RUS,Ford Focus,2.0,600.0,NaN


Посчитаем количество значений

In [4]:
df.count()

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64

Удалим повторные записи. Уникальными считаем сочетания значений в трех заданных столбцах, между двумя одинаковыми наблюдениями нужно выбрать последнее.

In [5]:
df = df.drop_duplicates(subset=['CarNumber', 'Make_n_model', 'Fines'], keep='last')
df.count()

CarNumber       725
Make_n_model    725
Refund          713
Fines           665
History          65
dtype: int64

Проверим, сколько значений отсутствует в каждом столбце

In [6]:
df.isna().sum()

CarNumber         0
Make_n_model      0
Refund           12
Fines            60
History         660
dtype: int64

Удалим все столбцы с более чем 500 отсутствующими значениями, используя фргумент thresh

Проверим, сколько осталось пропущенных значений в каждом столбце.

In [7]:
df = df.dropna(axis=1, thresh=500)
df.isna().sum()

CarNumber        0
Make_n_model     0
Refund          12
Fines           60
dtype: int64

Заменим все пропущенные значения в столбце Refund предыдущим значением в столбце Refund. Проверим пропущенные значения

In [8]:
df['Refund'].fillna(method='ffill', inplace=True)
df.isna().sum()

CarNumber        0
Make_n_model     0
Refund           0
Fines           60
dtype: int64

Заменим все пропущенные значения в столбце Fines средним значением этого столбца

In [9]:
df.Fines.fillna(value=df.Fines.dropna().mean(skipna=True), inplace=True)


Проверим есть ли пропущенные значение теперь

In [10]:
df.isna().sum()

CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64

Разделим наименование машины (Make_n_model) на марку и модель. Используем метод apply. 
Удалим столбец Make_n_model

In [11]:
df['Make'] = df['Make_n_model'].apply(lambda x: x.split()[0])
df['Model'] = df['Make_n_model'].apply(lambda x: x.split()[1] if len(x.split()) == 2 else None)
df.drop('Make_n_model', inplace=True, axis=1)
df

,CarNumber,Refund,Fines,Make,Model
ID,,,,,
0,Y163O8161RUS,2.0,3200.000000,Ford,Focus
1,E432XX77RUS,1.0,6500.000000,Toyota,Camry
2,7184TT36RUS,1.0,2100.000000,Ford,Focus
3,X582HE161RUS,2.0,2000.000000,Ford,Focus
5,92918M178RUS,1.0,5700.000000,Ford,Focus
...,...,...,...,...,...
926,Y163O8161RUS,2.0,1600.000000,Ford,Focus
927,M0309X197RUS,1.0,22300.000000,Ford,Focus
928,O673E8197RUS,2.0,600.000000,Ford,Focus


Сохраним датафрейм в файле JSON. 
Задаем ориентацию преобразования в строку (по дефолту columns) и количество знаков после запятой

In [12]:
df.to_json('../data/auto.json', orient='records', double_precision=1)

### Tests

In [13]:
df2 = pd.read_json('../data/auto.json', orient='records')
df2

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.0,Ford,Focus
1,E432XX77RUS,1,6500.0,Toyota,Camry
2,7184TT36RUS,1,2100.0,Ford,Focus
3,X582HE161RUS,2,2000.0,Ford,Focus
4,92918M178RUS,1,5700.0,Ford,Focus
...,...,...,...,...,...
720,Y163O8161RUS,2,1600.0,Ford,Focus
721,M0309X197RUS,1,22300.0,Ford,Focus
722,O673E8197RUS,2,600.0,Ford,Focus
723,8610T8154RUS,1,2000.0,Ford,Focus


In [14]:
df2.count()

CarNumber    725
Refund       725
Fines        725
Make         725
Model        716
dtype: int64

In [15]:
df2['Fines'].mean()

8594.587586206897

In [16]:
df2['Refund'].mean()

1.5172413793103448